# Bayesian Change-Point Analysis with PyMC

This notebook implements a structural break model for Brent oil price returns, matches detected shifts to external events, and provides rich visualizations.

In [ ]:
import pymc as pm
import arviz as az
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os

# 1. Load and prep data
df = pd.read_csv('../data/brent_daily.csv')
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df = df.sort_values('Date').reset_index(drop=True)
df['logP'] = np.log(df['Price'])
df['r'] = df['logP'].diff()
df = df.dropna().reset_index(drop=True)

y = df['r'].values
n = len(y)
print(f"Data loaded: {n} observations from {df['Date'].min().date()} to {df['Date'].max().date()}")

In [ ]:
# 2. PyMC Model Definition
with pm.Model() as cp_model:
    tau = pm.DiscreteUniform("tau", lower=0, upper=n-1)
    mu1 = pm.Normal("mu1", mu=0, sigma=0.01)
    mu2 = pm.Normal("mu2", mu=0, sigma=0.01)
    sigma = pm.HalfNormal("sigma", sigma=0.01)

    idx = np.arange(n)
    mu = pm.math.switch(idx <= tau, mu1, mu2)
    obs = pm.Normal("obs", mu=mu, sigma=sigma, observed=y)

    trace = pm.sample(draws=2000, tune=1000, chains=4, target_accept=0.95)

In [ ]:
# 3. Extraction of Key Metrics
tau_samples = trace.posterior['tau'].values.flatten()
tau_median = int(np.median(tau_samples))
cp_date = df['Date'].iloc[tau_median]
hdi_tau = az.hdi(tau_samples, hdi_prob=0.95)
cp_hdi_start_idx = int(hdi_tau[0])
cp_hdi_end_idx = int(hdi_tau[1])
cp_hdi_start_date = df['Date'].iloc[cp_hdi_start_idx]
cp_hdi_end_date = df['Date'].iloc[cp_hdi_end_idx]

mu1_samples = trace.posterior['mu1'].values.flatten()
mu2_samples = trace.posterior['mu2'].values.flatten()

events_df = pd.read_csv('../data/events.csv')
events_df.columns = [c.strip().lower() for c in events_df.columns]
events_df['date'] = pd.to_datetime(events_df['date'])

## Visualizations

In [ ]:
# A. Price Series with Change Point & Events
plt.figure(figsize=(15, 7))
plt.plot(df['Date'], df['Price'], label='Brent Price', color='black', alpha=0.7)

# Vertical Line for CP Median
plt.axvline(cp_date, color='red', linestyle='--', label=f'Median CP: {cp_date.date()}')

# Shaded Credible Interval Ribbon
plt.axvspan(cp_hdi_start_date, cp_hdi_end_date, color='red', alpha=0.2, label='95% CP Credible Interval')

# Mark Events
for i, row in events_df.iterrows():
    plt.axvline(row['date'], color='blue', alpha=0.3, linestyle=':')
    plt.text(row['date'], df['Price'].max()*0.9, row['title'], rotation=90, verticalalignment='center', fontsize=8, color='blue')

plt.title("Brent Crude Price with Detected Change Point & Historical Events")
plt.legend()
plt.show()

In [ ]:
# B. Log Returns with Shaded CP Period
plt.figure(figsize=(15, 5))
plt.plot(df['Date'], df['r'], color='gray', alpha=0.5, label='Daily Log Returns')
plt.axvspan(cp_hdi_start_date, cp_hdi_end_date, color='yellow', alpha=0.3, label='CP Uncertainty Window')
plt.axvline(cp_date, color='red', linestyle='--')
plt.title("Daily Log Returns & Change Point Window")
plt.legend()
plt.show()

In [ ]:
# C. Posterior Distribution of Tau
plt.figure(figsize=(10, 4))
plt.hist(tau_samples, bins=50, color='skyblue', edgecolor='black')
plt.axvline(tau_median, color='red', linestyle='-', label='Median Tau')
plt.title("Posterior Distribution of Change Point index (τ)")
plt.xlabel("Index")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# D. Mu1 vs Mu2 Density Plots
plt.figure(figsize=(10, 5))
sns.kdeplot(mu1_samples, shade=True, label='μ1 (Before CP)', color='green')
sns.kdeplot(mu2_samples, shade=True, label='μ2 (After CP)', color='orange')
prob_m2_gt_m1 = np.mean(mu2_samples > mu1_samples)
plt.text(np.mean(mu1_samples), 10, f"P(μ2 > μ1) = {prob_m2_gt_m1:.2%}", fontsize=12, fontweight='bold')
plt.title("Comparison of Posterior Means Before vs After Change Point")
plt.legend()
plt.show()

In [ ]:
# E. Rolling Volatility with Marker Events
df['vol30'] = df['r'].rolling(30).std() * np.sqrt(252)
plt.figure(figsize=(15, 5))
plt.plot(df['Date'], df['vol30'], label='30-day Rolling Vol (Annualized)', color='purple')

for i, row in events_df.iterrows():
    plt.scatter(row['date'], df['vol30'].max()*0.1, color='red', marker='x', alpha=0.8)

plt.title("Historical Volatility with Event Markers")
plt.legend()
plt.show()

## Operational Method: Ranking Events

In [ ]:
events_df['proximity_days'] = (events_df['date'] - cp_date).dt.days
events_df['abs_proximity'] = events_df['proximity_days'].abs()
candidates = events_df.sort_values('abs_proximity').head(5).copy()

mu1_mean, mu2_mean = np.mean(mu1_samples), np.mean(mu2_samples)
pct_change = (np.exp(mu2_mean) - np.exp(mu1_mean)) * 100

print(f"Detected Change Point Median: {cp_date.date()}")
print(f"95% Credible Interval: {cp_hdi_start_date.date()} to {cp_hdi_end_date.date()}")

print("\n--- Top Candidate Events Per CP ---")
for i, row in candidates.iterrows():
    relevance = "High" if any(k in str(row['type']).lower() for k in ['policy', 'conflict', 'sanctions']) else "Medium"
    print(f"- {row['title']} ({row['date'].date()})")
    print(f"  Distance: {row['proximity_days']} days. Relevance: {relevance}")
    print(f"  Notes: {row['notes']}")

In [ ]:
# 4. Export Results for Backend
summary = {
    "cp_median_date": str(cp_date.date()),
    "cp_hdi": [str(cp_hdi_start_date.date()), str(cp_hdi_end_date.date())],
    "mu1_mean": float(mu1_mean),
    "mu2_mean": float(mu2_mean),
    "pct_change": float(pct_change),
    "confidence": float(prob_m2_gt_m1 if mu2_mean > mu1_mean else (1 - prob_m2_gt_m1))
}

os.makedirs('../backend/results', exist_ok=True)
with open('../backend/results/change_point_summary.json', 'w') as f:
    json.dump(summary, f, indent=4)

print("\nSummary results exported to backend/results/change_point_summary.json")